In [390]:
import pandas as pd

In [391]:
df = pd.read_csv('glassdoor_jobs.csv')

In [392]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


#salary:
    1. Min & Max separate columns
    2. Remove glassdoor est
#Company Name
    1. Keep name only, remove rating
#Location
    1. City and State separate
#Founded
    1. Company Age
#Job Description
    1. Parse JD for keywords

In [394]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         956 non-null    int64  
 1   Job Title          956 non-null    object 
 2   Salary Estimate    956 non-null    object 
 3   Job Description    956 non-null    object 
 4   Rating             956 non-null    float64
 5   Company Name       956 non-null    object 
 6   Location           956 non-null    object 
 7   Headquarters       956 non-null    object 
 8   Size               956 non-null    object 
 9   Founded            956 non-null    int64  
 10  Type of ownership  956 non-null    object 
 11  Industry           956 non-null    object 
 12  Sector             956 non-null    object 
 13  Revenue            956 non-null    object 
 14  Competitors        956 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


In [395]:
df.isna().sum()

Unnamed: 0           0
Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
Competitors          0
dtype: int64

In [396]:
(df['Salary Estimate'] == '-1').sum()

214

In [397]:
df = df[df['Salary Estimate']!='-1']

# Regex

### Cleaning Salary

In [398]:
regex_df = df.copy()

In [399]:
#Glassdoor estimate removal
salary = regex_df['Salary Estimate'].apply(lambda x:x.split('(')[0])

0                                     $53K-$91K 
1                                    $63K-$112K 
2                                     $80K-$90K 
3                                     $56K-$97K 
4                                    $86K-$143K 
5                                    $71K-$119K 
6                                     $54K-$93K 
7                                    $86K-$142K 
8                                     $38K-$84K 
9                                   $120K-$160K 
10                                  $126K-$201K 
11                                   $64K-$106K 
12                                  $106K-$172K 
13                                    $46K-$85K 
14                                   $83K-$144K 
15                                  $102K-$190K 
16                                   $67K-$137K 
17                                  $118K-$189K 
18                                  $110K-$175K 
19                                   $64K-$111K 
20                  

In [400]:
regex_df['per_hour'] = salary.apply(lambda x: 1 if "per hour" in x.lower() else 0)

In [401]:
regex_df['employer_provided'] = salary.apply(lambda x: 1 if "employer provided salary:" in x.lower() else 0)

In [402]:
#'k' and '$' removal
remove_kd = salary.apply(lambda x:x.replace('K','').replace('$', ''))

0                                       53-91 
1                                      63-112 
2                                       80-90 
3                                       56-97 
4                                      86-143 
5                                      71-119 
6                                       54-93 
7                                      86-142 
8                                       38-84 
9                                     120-160 
10                                    126-201 
11                                     64-106 
12                                    106-172 
13                                      46-85 
14                                     83-144 
15                                    102-190 
16                                     67-137 
17                                    118-189 
18                                    110-175 
19                                     64-111 
20                                     81-130 
21           

In [403]:
remove_text = remove_kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', ''))

0        53-91 
1       63-112 
2        80-90 
3        56-97 
4       86-143 
5       71-119 
6        54-93 
7       86-142 
8        38-84 
9      120-160 
10     126-201 
11      64-106 
12     106-172 
13       46-85 
14      83-144 
15     102-190 
16      67-137 
17     118-189 
18     110-175 
19      64-111 
20      81-130 
21      73-119 
22      86-139 
23      63-105 
25     109-177 
26      63-110 
27      75-124 
28       34-61 
29      63-105 
30       80-90 
31       56-97 
32      72-120 
33      86-143 
34      93-149 
35      85-140 
36      77-135 
37      82-132 
38      83-137 
39     115-180 
40      74-138 
41      64-112 
43      68-129 
44     110-175 
45      52-113 
46      110-150
48      150-160
49     158-211 
50       20-39 
51      56-117 
52       63-99 
54      68-114 
55      86-143 
56       41-95 
57      86-144 
58      80-139 
59       56-95 
60     120-189 
61     111-176 
62       54-93 
63      71-119 
65      82-132 
66      84-146 
67     1

In [404]:
regex_df['min salary'] = remove_text.apply(lambda x: x.split('-')[0])

In [405]:
regex_df['max_salary'] =  remove_text.apply(lambda x:x.split('-')[1])

### Company Name Parse

In [407]:
remove_n = regex_df['Company Name'].apply(lambda x: x.replace('\n', ','))

0                                  Tecolote Research,3.8
1              University of Maryland Medical System,3.4
2                                            KnowBe4,4.8
3                                               PNNL,3.8
4                                 Affinity Solutions,2.9
5                                           CyrusOne,3.4
6                                 ClearOne Advantage,4.1
7                                         Logic20/20,3.8
8                          Rochester Regional Health,3.3
9                                           <intent>,4.6
10                                              Wish,3.5
11                                           ManTech,4.1
12                                           Walmart,3.2
13                                            Yesler,4.1
14                            Takeda Pharmaceuticals,3.7
15                                           Audible,3.6
16                               Blueprint Medicines,3.9
17                             

In [408]:
regex_df['company_txt'] = remove_n.apply(lambda x: x.split(',')[0])

# Location

In [409]:
extract_state = regex_df['Location'].apply(lambda x: x.split(',')[1])
extract_state = extract_state.apply(lambda x: x.replace('Los Angeles', 'CA'))

0       NM
1       MD
2       FL
3       WA
4       NY
5       TX
6       MD
7       CA
8       NY
9       NY
10      CA
11      VA
12      TX
13      WA
14      MA
15      NJ
16      MA
17      CA
18      CA
19      CO
20      WA
21      TX
22      IL
23      KY
25      VA
26      MA
27      OR
28      MA
29      CT
30      FL
31      WA
32      MI
33      MA
34      CA
35      MA
36      CA
37      CA
38      MA
39      CA
40      NY
41      CA
43      MA
44      CA
45      NJ
46      IL
48      DC
49      WA
50      CO
51      OH
52      IL
54      FL
55      MA
56      AL
57      NY
58      CA
59      NM
60      CA
61      MO
62      MD
63      TX
65      OH
66      NY
67      CA
68      WA
70      PA
72      MA
73      GA
75      MA
76      PA
77      WA
78      IN
79      IL
80      MO
81      VA
82      PA
83      WA
84      CA
85      MA
86      VA
87      CA
88      CA
89      PA
90      MD
91      IL
92      VA
93      WA
94      TX
95      CA
96      FL
97      AL
99      LA

In [410]:
extract_city = regex_df['Location'].apply(lambda x: x.split(',')[0])

0              Albuquerque
1                Linthicum
2               Clearwater
3                 Richland
4                 New York
5                   Dallas
6                Baltimore
7                 San Jose
8                Rochester
9                 New York
10                San Jose
11               Chantilly
12                   Plano
13                 Seattle
14               Cambridge
15                  Newark
16               Cambridge
17           Mountain View
18           San Francisco
19                  Denver
20                 Seattle
21                  Dallas
22                 Chicago
23              Louisville
25                 Herndon
26               Cambridge
27               Hillsboro
28               Worcester
29                  Groton
30              Clearwater
31                Richland
32                 Detroit
33               Cambridge
34               Sunnyvale
35                 Ipswich
36                San Jose
37                Redlands
3

In [411]:
regex_df['State'] = extract_state
regex_df['City'] = extract_city

In [412]:
# It was throwing an error with x.split(',')[1], so used [-1] to get the states values.
extract_hq_state = regex_df['Headquarters'].apply(lambda x: x.split(',')[-1])

0                   CA
1                   MD
2                   FL
3                   WA
4                   NY
5                   TX
6                   MD
7                   WA
8                   NY
9                   NY
10                  CA
11                  VA
12                  AR
13                  WA
14               Japan
15                  NJ
16                  MA
17                  CA
18                  CA
19                  CO
20                  WA
21                  CA
22                  ID
23                  GA
25                  VA
26                  NY
27                  CA
28                  MA
29                  NY
30                  FL
31                  WA
32                  MI
33         Switzerland
34                  CA
35                  MA
36                  IL
37                  CA
38                  MA
39             Germany
40                  NY
41                  CA
43                  MA
44                  CA
45         

In [413]:
#Test since above code wasn't waorking, it worked with [-1]
for x in regex_df['Headquarters']:
    split_result = x.split(',')[1]
    print((split_result))

 CA
 MD
 FL
 WA
 NY
 TX
 MD
 WA
 NY
 NY
 CA
 VA
 AR
 WA
 Japan
 NJ
 MA
 CA
 CA
 CO
 WA
 CA
 ID
 GA
 VA
 NY
 CA
 MA
 NY
 FL
 WA
 MI
 Switzerland
 CA
 MA
 IL
 CA
 MA
 Germany
 NY
 CA
 MA
 CA
 CT
 IL
 VA
 CA
 NE
 VA
 IL
 FL
 MA
 AL
 Switzerland
 VA
 Germany
 CA
 MO
 MD
 TX
 OH
 NY
 CA
 WA
 PA
 MA
 GA
 MA
 PA
 CA
 CA
 IL
 CA
 VA
 Belgium
 CA
 CA
 MA
 VA
 China
 CA
 PA
 MD
 IL
 VA
 WA
 AR
 WA
 FL
 VA
 LA
 CA
 IL
 VA
 GA
 VA
 MA
 VA
 MA
 MD
 NY
 NY
 NY
 DC
 VA
 MI
 FL
 AZ
 NE
 CA
 NJ
 Japan
 NY
 CA
 CA
 IN
 MD
 MN
 CA
 SC
 MA
 OH
 DC
 IL
 MD
 AL
 CA
 MO
 CA
 IL
 VA
 TX
 CA
 CA
 CO
 United Kingdom
 VA
 CA
 VA
 IL
 CO
 UT
 CA
 PA
 WA
 WA
 CA
 OH
 TN
 WA
 FL
 IN
 MA
 NJ
 MA
 CA
 CA
 Japan
 MD
 CA
 CA
 CA
 UT
 MA
 VA
 PA
 WI
 MA
 CA
 MI
 FL
 CA
 Japan
 NJ
 Japan
 MA
 CA
 DE
 Japan
 Bermuda
 MO
 PA
 CA
 CA
 VA
 MA
 MA
 IL
 UT
 CO
 NY
 MD
 MA
 CA
 NY
 CA
 CA
 TX
 TX
 PA
 CA
 TN
 DE
 WI
 MD
 IL
 ID
 MD
 WI
 NY
 DC
 TX
 FL
 NJ
 MA
 IL
 South Korea
 MA
 OH
 MA
 CA
 CA
 CA
 Japan
 CA
 OH
 CA
 MD
 NJ
 

IndexError: list index out of range

In [414]:
temp_df = pd.DataFrame({'extract_state' : extract_state, 'extract_hq_state' : extract_hq_state})


In [416]:
same_state_hq = temp_df.apply(lambda x: 1 if x.extract_state == x.extract_hq_state else 0, axis =1)


In [417]:
regex_df['same_state_hq'] = same_state_hq

# Founded

In [418]:
from datetime import datetime

In [419]:
year = datetime.now().year

In [420]:
company_age = []
for x in regex_df.Founded:
    if x == -1:
        company_age.append('NAN')
    else:
        age = year - x
        company_age.append(age)


In [421]:
regex_df['company_age'] = company_age

# JD Parse (Added each of these series to regex_df separately)

In [422]:
regex_df['jd_python'] = regex_df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() or 'py' in x.lower() else 0)

In [423]:
jd_python.value_counts()

1    418
0    324
Name: Job Description, dtype: int64

In [424]:
regex_df['jd_sql'] = regex_df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() or 'mysql' in x.lower() else 0)

In [425]:
jd_sql.value_counts()

1    380
0    362
Name: Job Description, dtype: int64

In [428]:
regex_df['jd_excel'] = regex_df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() or 'msexcel' in x.lower() else 0)

In [429]:
jd_excel.value_counts()

1    388
0    354
Name: Job Description, dtype: int64

In [430]:
regex_df['jd_spark'] = regex_df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [431]:
jd_spark.value_counts()

0    575
1    167
Name: Job Description, dtype: int64

In [432]:
regex_df['jd_apache'] = regex_df['Job Description'].apply(lambda x: 1 if 'apache' in x.lower() else 0)

In [433]:
jd_apache.value_counts()

0    709
1     33
Name: Job Description, dtype: int64

In [434]:
regex_df['jd_r'] = regex_df['Job Description'].apply(lambda x: 1 if 'r' in x.lower() or ' r studio' in x.lower() or 'r-studio' in x.lower()  else 0)

In [435]:
jd_r.value_counts()

1    742
Name: Job Description, dtype: int64

In [436]:
regex_df['jd_tensor'] = regex_df['Job Description'].apply(lambda x: 1 if 'tensorflow' in x.lower() or 'tensor' in x.lower() or 'tensor flow' in x.lower() else 0)

In [437]:
jd_tensor.value_counts()

0    670
1     72
Name: Job Description, dtype: int64

In [438]:
regex_df['jd_jupyter'] = regex_df['Job Description'].apply(lambda x: 1 if 'jupyter' in x.lower() or 'notebook' in x.lower() else 0)

In [439]:
jd_jupyter.value_counts()

0    697
1     45
Name: Job Description, dtype: int64

In [440]:
regex_df['jd_git'] = regex_df['Job Description'].apply(lambda x: 1 if 'git' in x.lower() or 'github' in x.lower() else 0)

In [441]:
jd_git.value_counts()

0    581
1    161
Name: Job Description, dtype: int64

In [442]:
regex_df['jd_library'] = regex_df['Job Description'].apply(lambda x: 1 if 'pandas' in x.lower() or 'numpy' in x.lower() or 'scikit' in x.lower() or 'scikit-learn' in x.lower() else 0)

In [443]:
jd_library.value_counts()

0    672
1     70
Name: Job Description, dtype: int64

In [444]:
regex_df['jd_aws'] = regex_df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [445]:
jd_aws.value_counts()

0    566
1    176
Name: Job Description, dtype: int64

In [446]:
regex_df['jd_cloud'] = regex_df['Job Description'].apply(lambda x: 1 if 'cloud' in x.lower() or 'google cloud' in x.lower() else 0)

In [447]:
jd_cloud.value_counts()

0    576
1    166
Name: Job Description, dtype: int64

In [448]:
regex_df['jd_linux'] = regex_df['Job Description'].apply(lambda x: 1 if 'linux' in x.lower() else 0)

In [449]:
jd_linux.value_counts()

0    686
1     56
Name: Job Description, dtype: int64

In [451]:
regex_df = regex_df.drop(['Unnamed: 0'], axis = 1)

# Job Title Parse

In [501]:
clean_title = []
for x in regex_df['Job Title']:
    match = re.findall(r'\bdata scientist\b', x, flags = re.IGNORECASE)
    match_2 = re.findall(r'\bdata science\b', x, flags = re.IGNORECASE)
    match_3 = re.findall(r'\bdata analyst\b', x, flags = re.IGNORECASE)
    match_4 = re.findall(r'\bdata analytics\b', x, flags = re.IGNORECASE)
    match_5 = re.findall(r'\bdata engineer\b', x, flags = re.IGNORECASE)
    match_6 = re.findall(r'\bresearch scientist\b', x, flags = re.IGNORECASE)
    match_7 = re.findall(r'\bmachine learning\b', x, flags = re.IGNORECASE)
    match_8 = re.findall(r'\bdirector\b', x, flags = re.IGNORECASE)
    match_9 = re.findall(r'\bmanager\b', x, flags = re.IGNORECASE)
    match_10 = re.findall(r'\bconsultant\b', x, flags = re.IGNORECASE)
    match_11 = re.findall(r'\bassociate director\b', x, flags = re.IGNORECASE)
    match_12 = re.findall(r'\banalyst\b', x, flags = re.IGNORECASE)
    match_13 = re.findall(r'\bdata engineering\b', x, flags = re.IGNORECASE)
    if match or match_2:
        clean_title.append(x.replace(x, 'Data Scientist'))
        #regex_df[job_title] = job_title.append(x.replace(x, 'Data Scientist'))
    elif match_3:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_4:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_5:
        clean_title.append(x.replace(x, 'Data Engineer'))
    elif match_6:
        clean_title.append(x.replace(x, 'Research Scientist'))
    elif match_7:
        clean_title.append(x.replace(x, 'Machine Learning Engineer'))
    elif match_8:
        clean_title.append(x.replace(x, 'Director'))
    elif match_9:
        clean_title.append(x.replace(x, 'Manager'))
    elif match_10:
        clean_title.append(x.replace(x, 'Consultant'))
    elif match_11:
        clean_title.append(x.replace(x, 'Associate Director'))
    elif match_12:
        clean_title.append(x.replace(x, 'Data Analyst'))
    elif match_13:
        clean_title.append(x.replace(x, 'Data Engineer'))
    else:
        clean_title.append(x.replace(x, 'NA'))

In [502]:
regex_df['clean_title'] = clean_title

In [503]:
regex_df.clean_title.value_counts()

Data Scientist               313
NA                           143
Data Engineer                115
Data Analyst                 106
Research Scientist            22
Machine Learning Engineer     15
Manager                       13
Director                       8
Consultant                     7
Name: clean_title, dtype: int64

In [504]:
regex_df.clean_title.value_counts().sum()

742

In [485]:
regex_df.clean_title.value_counts().to_csv('Clean_titles.csv', index = False)

# JD Length

In [506]:
regex_df['jd_length'] = regex_df['Job Description'].apply(lambda x: len(x))

# Competitor Count

In [512]:
regex_df['comp_count'] = regex_df['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)

# Per hour to annual


In [536]:
regex_df['min salary'] = pd.to_numeric(regex_df['min salary'], errors='coerce')
regex_df['max_salary'] = pd.to_numeric(regex_df['max_salary'], errors='coerce')

In [537]:
regex_df['min salary'] = regex_df.apply(lambda x: round(x['min salary']*1.92,0) if x.per_hour == 1 else x['min salary'], axis =1)

In [538]:
regex_df['max_salary'] = regex_df.apply(lambda x: round(x['max_salary']*1.92,0) if x.per_hour == 1 else x['max_salary'], axis =1)

In [540]:
regex_df

,Job Title,clean_title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,per_hour,employer_provided,min salary,max_salary,company_txt,State,City,same_state_hq,company_age,jd_python,jd_sql,jd_excel,jd_spark,jd_apache,jd_r,jd_tensor,jd_jupyter,jd_git,jd_library,jd_aws,jd_cloud,jd_linux,jd_length,comp_count
0,Data Scientist,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53.0,91.0,Tecolote Research,NM,Albuquerque,0,50,1,0,1,0,0,1,0,0,0,0,0,0,0,2536,0
1,Healthcare Data Scientist,Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63.0,112.0,University of Maryland Medical System,MD,Linthicum,1,39,1,0,0,0,0,1,0,0,0,0,0,0,0,4783,0
2,Data Scientist,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80.0,90.0,KnowBe4,FL,Clearwater,1,13,1,1,1,1,0,1,0,0,0,0,0,0,0,3461,0
3,Data Scientist,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56.0,97.0,PNNL,WA,Richland,1,58,1,0,0,0,0,1,0,0,0,0,0,0,0,3883,3
4,Data Scientist,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86.0,143.0,Affinity Solutions,NY,New York,1,25,1,1,1,0,0,1,0,0,0,0,0,1,0,2728,3
5,Data Scientist,Data Scientist,$71K-$119K (Glassdoor est.),CyrusOne is seeking a talented Data Scientist ...,3.4,CyrusOne\n3.4,"Dallas, TX","Dallas, TX",201 to 500 employees,2000,Company - Public,Real Estate,Real Estate,$1 to $2 billion (USD),"Digital Realty, CoreSite, Equinix",0,0,71.0,119.0,CyrusOne,TX,Dallas,1,23,1,1,1,0,0,1,0,0,1,0,1,0,0,3747,3
6,Data Scientist,Data Scientist,$54K-$93K (Glassdoor est.),Job Description\n\n**Please only local candida...,4.1,ClearOne Advantage\n4.1,"Baltimore, MD","Baltimore, MD",501 to 1000 employees,2008,Company - Private,Banks & Credit Unions,Finance,Unknown / Non-Applicable,-1,0,0,54.0,93.0,ClearOne Advantage,MD,Baltimore,1,15,0,0,1,0,0,1,0,0,0,0,0,0,0,1786,0
7,Data Scientist,Data Scientist,$86K-$142K (Glassdoor est.),Advanced Analytics – Lead Data Scientist\nOver...,3.8,Logic20/20\n3.8,"San Jose, CA","Seattle, WA",201 to 500 employees,2005,Company - Private,Consulting,Business Services,$25 to $50 million (USD),-1,0,0,86.0,142.0,Logic20/20,CA,San Jose,0,18,1,1,1,1,0,1,1,0,0,0,1,0,0,3804,0
8,Research Scientist,Research Scientist,$38K-$84K (Glassdoor est.),SUMMARY\n\nThe Research Scientist I will be ta...,3.3,Rochester Regional Health\n3.3,"Rochester, NY","Rochester, NY",10000+ employees,2014,Hospital,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),-1,0,0,38.0,84.0,Rochester Regional Health,NY,Rochester,1,9,0,0,0,0,0,1,0,0,0,0,0,0,0,1538,0
9,Data Scientist,Data Scientist,$120K-$160K (Glassdoor est.),isn’t your usual company. Our work is powered ...,4.6,<intent>\n4.6,"New York, NY","New York, NY",51 to 200 employees,2009,Company - Private,Internet,Information Technology,$100 to $500 million (U